In [ ]:
import pandas as pd
import warnings
import re
import folium
from selenium import webdriver
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_parquet("FINAL_FLORIDA.parquet")


In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
# Suponiendo que tu DataFrame se llama 'joined'
unique_addresses = df.drop_duplicates(subset='address', keep='first')

In [ ]:
# Crea un mapa centrado alrededor de Florida
florida_map = folium.Map(location=[27.9944024, -81.7602544], zoom_start=6) 

# Agrega cada punto del dataframe al mapa
for _, row in unique_addresses.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        fill=True,
        fill_color="blue",
        fill_opacity=0.6
    ).add_to(florida_map)

florida_map
#tardo 5 minutos 24 segundos
florida_map.save("mapa_restaurants.html")


In [ ]:
nan_count = df['address'].isna().sum()
print(f"Number of NaN values in 'address' column: {nan_count}")


In [ ]:
df.head(1)

In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd

# Contar las repeticiones de cada par de coordenadas
df['coord_count'] = df.groupby(['latitude', 'longitude']).transform('size')

# Crear un mapa centrado en Florida
m = folium.Map(location=[27.9944024, -81.7602544], zoom_start=6)  # Estas coordenadas centran el mapa en Florida

# Preparar los datos para el heatmap
# Las filas con mayor conteo tendrán mayor peso en el heatmap
heat_data = [[row['latitude'], row['longitude'], row['coord_count']] for index, row in df.iterrows()]

# Agregar el heatmap al mapa
HeatMap(heat_data,radius=5,blur=3).add_to(m)

m

m.save("mapa_de_calor_densidad_de_restaurants.html")

#tarda 

In [ ]:
import numpy as np
import folium
from folium.plugins import HeatMap

# Contar las repeticiones de cada combinación de latitud y longitud
df['latlong_count'] = df.groupby(['latitude', 'longitude']).transform('count').iloc[:, 0]

# Transformar usando escala logarítmica. Agregamos 1 para evitar log(0).
df['log_count'] = np.log(df['latlong_count'] + 1)

# Crear un mapa centrado en Florida
m = folium.Map(location=[27.9944024, -81.7602544], zoom_start=7)

# Preparar los datos para el heatmap
heat_data = [[row['latitude'], row['longitude'], row['log_count']] for index, row in df.iterrows()]

# Definir el gradiente de color
gradient = {
    0.0: '#0000ff',
    0.5: '#00ff00',
    1.0: '#ff0000'
}

# Agregar el heatmap al mapa
HeatMap(heat_data, gradient=gradient, radius=5,blur=5).add_to(m)

m.save("mapa_restaurants_densidad_mas_en_detalle.html")


m


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Extraer el año de la columna 'time'
df['year'] = pd.to_datetime(df['time'], format='%d/%m/%Y').dt.year

# Agrupar por latitud, longitud y año, y contar duplicados
grouped = df.groupby(['latitude', 'longitude', 'year']).size().reset_index(name='duplicates_count')

# Sumar los duplicados por año
sum_duplicados_by_year = grouped.groupby('year')['duplicates_count'].sum()

# Graficar
sum_duplicados_by_year.plot(kind='bar', figsize=(10, 6))
plt.title('Sumatoria de reviews por año')
plt.xlabel('Año')
plt.ylabel('Sumatoria de reviews por año')
plt.xticks(rotation=45)
plt.tight_layout()

# Guardar la figura en PNG
plt.savefig("sumatoria_de_reviews_por_anio.png", dpi=300)

plt.show()


In [ ]:
df.shape

In [ ]:
import pandas as pd

# Convertir la columna 'time' a formato de fecha
df['time'] = pd.to_datetime(df['time'], format='%d/%m/%Y')

# Extraer el año de la columna 'time'
df['year'] = df['time'].dt.year

# Filtrar el DataFrame por las filas en el rango de años 2018-2021
filtered_df = df[df['year'].between(2018, 2021)]

filtered_df.shape

In [ ]:
filtered_df.head(1)

In [ ]:
from textblob import TextBlob

# Función para determinar el nivel de sentimiento de una serie de textos
def sentiment_level(text_series):
    polarities = []

    for text in text_series:
        analysis = TextBlob(text)
        polarities.append(analysis.sentiment.polarity)

    # Calcula la polaridad promedio de los textos
    avg_polarity = sum(polarities) / len(polarities)

    # Clasificar el nivel de sentimiento basado en la polaridad promedio
    if avg_polarity <= -0.2:
        return 1  # Negativo
    elif -0.2 < avg_polarity <= -0.1:
        return 2  # No tan negativo
    elif -0.1 < avg_polarity < 0.1:
        return 3  # Neutral
    elif 0.1 <= avg_polarity < 0.4:
        return 4  # Positivo
    else:
        return 5  # Muy positivo

# Agrupar por año, latitud y longitud y calcular el nivel de sentimiento para cada grupo
sentiment_levels_by_group = df.groupby(['year', 'latitude', 'longitude'])['text'].apply(sentiment_level).reset_index()

# Reformatar y renombrar las columnas para facilitar la interpretación
sentiment_levels_by_group.columns = ['year', 'latitude', 'longitude', 'Sentiment Level']

# Fusiona df con sentiment_levels_by_group para agregar la columna 'Sentiment Level'
df = pd.merge(df, sentiment_levels_by_group, on=['year', 'latitude', 'longitude'], how='left')




In [ ]:
# Usar value_counts() para obtener una serie de valores y sus conteos.
value_counts = df['Sentiment Level'].value_counts()

# Valor máximo
max_val = df['Sentiment Level'].max()

# Valor mínimo
min_val = df['Sentiment Level'].min()

# Primera moda (valor más repetido)
first_mode_val = value_counts.index[0]

# Segunda moda (segundo valor más repetido)
second_mode_val = value_counts.index[1] if len(value_counts) > 1 else None

# Tercera moda (tercer valor más repetido)
third_mode_val = value_counts.index[2] if len(value_counts) > 2 else None

# Promedio
average_val = df['Sentiment Level'].mean()

print(f"Máximo: {max_val}")
print(f"Mínimo: {min_val}")
print(f"1er Moda (valor más repetido): {first_mode_val}")
print(f"2do Moda (segundo valor más repetido): {second_mode_val}")
print(f"3er Moda (tercer valor más repetido): {third_mode_val}")
print(f"Promedio: {average_val}")



In [ ]:

import folium
import numpy as np
import pandas as pd

# Función para determinar el color basado en el nivel de sentimiento
def sentiment_color(sentiment_level):
    colors = {
        1: '#FF0000',   # rojo
        2: '#FFA500',   # naranja
        3: '#FFFF00',   # amarillo
        4: '#7CFC00',   # verde claro
        5: '#006400'    # verde oscuro
    }
    return colors.get(sentiment_level, '#FFFFFF')  # blanco por defecto para cualquier otro caso

# Función para la tendencia de sentimientos
def sentiment_trend(sentiments):
    if len(sentiments) == 1:
        return sentiments.iloc[0]
    # Si hay más de un año, calcular la tendencia
    average_sentiment = round(np.mean(sentiments))
    return average_sentiment

# Obtiene el nivel de tendencia del sentimiento
df['Trend Level'] = df.groupby(['latitude', 'longitude'])['Sentiment Level'].transform(sentiment_trend)

# Obtiene el color basado en el nivel de tendencia
df['color'] = df['Trend Level'].apply(sentiment_color)



In [ ]:

# Contar cuántos registros hay para cada combinación de latitud y longitud.
counts = df.groupby(['latitude', 'longitude']).size().reset_index(name='counts')

# Normalizar estos conteos para obtener una opacidad entre 0.5 y 1.
max_count = counts['counts'].max()
counts['opacity'] = 0.5 + 0.5 * (counts['counts'] / max_count)  # Ajusta según tus preferencias

# Unir estos valores de opacidad al DataFrame original.
df = df.merge(counts[['latitude', 'longitude', 'opacity']], on=['latitude', 'longitude'], how='left')

# Ordenar los datos en función del color
color_order = {'#006400': 5, '#7CFC00': 4, '#FFFF00': 3, '#FFA500': 2, '#FF0000': 1}
df['color_rank'] = df['color'].map(color_order)
df_sorted = df.sort_values(by='color_rank', ascending=False)

# Luego de esto, puedes proceder con la eliminación de duplicados como antes.
unique_locs = df_sorted.drop_duplicates(subset=['latitude', 'longitude', 'color', 'opacity'])


In [ ]:

# Crear mapa centrado en Florida
m = folium.Map(location=[27.9944024, -81.7602544] ,zoom_start=6)

for idx, row in unique_locs.iterrows():
    if pd.notna(row['color']):  # Solo dibujar si tenemos un color válido
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            fill_opacity=row['opacity'],
            weight=0,
            radius=2,
            blur=100
        ).add_to(m)

m.save("mapa_de_evolucion_de_sentimiento_3_a_anios.html")

m



In [ ]:
green_dark_coords = df[df['color'] == '#006400'][['latitude', 'longitude']].values.tolist()
yellow_coords = df[df['color'] == '#FFFF00'][['latitude', 'longitude']].values.tolist()
red_coords = df[df['color'] == '#FF0000'][['latitude', 'longitude']].values.tolist()

In [ ]:
import folium
from folium.plugins import HeatMap

m = folium.Map(location=[27.9944024, -81.7602544], zoom_start=6)

# Añade capa de calor para verde oscuro
HeatMap(green_dark_coords, name='Verde Oscuro', gradient={0: '#FFFFFF', 1: '#006400'}, radius=10, blur=5).add_to(m)

# Añade capa de calor para amarillo
HeatMap(yellow_coords, name='Amarillo', gradient={0: '#FFFFFF', 1: '#FFFF00'}, radius=6, blur=3).add_to(m)

# Añade capa de calor para rojo
HeatMap(red_coords, name='Rojo', gradient={0: '#FFFFFF', 1: '#FF0000'}, radius=10, blur=5).add_to(m)

# Opción para encender/apagar capas
folium.LayerControl().add_to(m)

m.save("mapa_de_densidad_de sentimiento_a_3_anios.html")


m
